In [27]:
import numpy as np
import os
import pandas as pd
from numpy.ma.extras import average
"""_vehicle_output 파일에서 데이터 가공 (차종별 평균, 가중 평균속도)"""
folder_path = r"C:\VISSIM_Workspace\network_new\test"
mer_name = "network01_mer"

file_path = os.path.join(folder_path, mer_name, "vehicle_output")
df = pd.read_excel(os.path.join(file_path, "1_vehicle_output.xlsx"))
df["NewMeasurem"] = df["Measurem."] % 1000

vehicle_type_list = [100, 300, 630, 640, 650]
results = []
for i in range(len(vehicle_type_list)):
    filtered_df = df[df["Vehicle type"] == 650]
    # 피벗 테이블 생성
    old_df = filtered_df.pivot_table(
        index="time",
        columns=["NewMeasurem", "Measurem."],
        values="v[km/h]",
        aggfunc="mean"
    )
    # 결과를 저장할 딕셔너리
    combined_cols = {}

    # level=0("NewMeasurem") 별로 그룹핑 후 2개 열의 데이터를 이어 붙인 후 새로운 컬럼 생성
    for measurem_group, cols in old_df.groupby(axis=1, level=0):
        if len(cols.columns) == 2:
            col1, col2 = cols.columns
            new_col_name = f'{col1[1]}'

            # 1번째 열과 2번째 열을 하나로 합친다.
            combined_series = pd.concat([cols[col1], cols[col2]], ignore_index=True)
            combined_cols[new_col_name] = combined_series
    combined_df = pd.DataFrame(combined_cols)


    # 평균속도
    avg_speed = combined_df.mean(axis=0)

    # 표준편차
    std = combined_df.std(ddof=0)
    # 변동계수
    vrt_num = std / avg_speed
    # 가중치
    add_per =  1/vrt_num

    for j in range(len(avg_speed)):
        results.append({
            "Vehicle type": vehicle_type_list[i],
            "NewMeasurem": int(list(avg_speed.index)[j]) % 1000,
            "AverageSpeed": avg_speed.iloc[j],
            "Weight (가중치)": add_per.iloc[j]
        })
weights_df = pd.DataFrame(results)
####### 가중 평균속도 계산 #######

# 평균속도 * 가중치
weights_df["Weighted Value"] = weights_df["AverageSpeed"] * weights_df["Weight (가중치)"]

# NewMeasurem별로 그룹화 및 합 계산
sum_weighted = weights_df.groupby("NewMeasurem")["Weighted Value"].sum()
sum_weights = weights_df.groupby("NewMeasurem")["Weight (가중치)"].sum()

# 가중 평균 속도 계산
final_df = (sum_weighted / sum_weights).reset_index(name="Weighted Avg Speed")
final_df = final_df.set_index("NewMeasurem").T
final_df.index = ["가중평균속도"]  # 이게 첫 번째 행(인덱스)로 들어감

print(final_df)


# 차종별 평균속도 계산 (NewMeasurem 기준)
vehicle_avg = weights_df.groupby(["NewMeasurem", "Vehicle type"])["AverageSpeed"].mean().unstack()
vehicle_avg = vehicle_avg.reindex(columns=vehicle_type_list)  # 차종 순서 고정
vehicle_avg = vehicle_avg.round(2)
vehicle_avg.index.name = "NewMeasurem"
vehicle_avg.columns = [f"차종{col}" for col in vehicle_avg.columns]

# 가중평균속도와 결합
final_df = pd.concat([vehicle_avg.T, final_df])
excel_dir = os.path.join(folder_path, mer_name, "weights_output")

NewMeasurem       1          2          3          4          5          6    \
가중평균속도       87.53913  87.817391  87.104545  87.415909  87.176087  87.415909   

NewMeasurem        7        8        9          10   ...      111        112  \
가중평균속도       87.445652  87.3375  87.3375  87.443478  ...  87.0875  86.895238   

NewMeasurem    113    114   115      116        117        118       119  \
가중평균속도       87.13  86.97  87.0  87.1675  87.854545  87.033333  86.87381   

NewMeasurem     120  
가중평균속도       86.709  

[1 rows x 119 columns]
